In [ ]:
#https://datascience.stackexchange.com/questions/40067/confusion-matrix-three-classes-python/40068 
#confusion matrix graphics code
import itertools
import matplotlib.pyplot as plt
import numpy as np


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True Growth')
    plt.xlabel('Predicted Growth')
    plt.tight_layout()

In [ ]:
#random forest code
import pandas as pd

#format input
input1= 'Y1000KEGG_m.csv'
input2='final_KEGG_carb.csv'

df1 = pd.read_csv(input1, index_col=0)
df2 = pd.read_csv(input2, index_col=0)
df = pd.concat([df1, df2.set_index(df1.index)], axis=1)


#import modules needed
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBRFClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.utils import compute_sample_weight
import matplotlib.pyplot as plt
from collections import Counter

#define what we want to predict out of the dataset
predicting = 'Carb_FINAL'
#drop NAs (species we didn't measure specialism/generalism) from the dataset
df = df.dropna(subset=[predicting])
#drop species that we did measure specialism/generalism but fall in the "normal" category and are not specialists or generalists 
df = df[df[predicting] != 0]
df[predicting] = df[predicting].replace([2],0)

#decide what other data you want to drop from the input, here I am dropping the predicted value
ml_input = np.array(input_ed) #array of input
output = np.array(df[predicting]) #array of output
print(output)

#build your model 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)

#use cross_val_predict to test the model and build the big confusion matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(model, ml_input, output, cv=10)
conf_mat = confusion_matrix(output, y_pred)
print(conf_mat)
plt.rcParams["figure.figsize"] = (3,3)
plt.figure()
plot_confusion_matrix(conf_mat, classes=['0','1'],
                     title='Confusion matrix')
plt.savefig("Confusion_Matrix.png", dpi=200)


#look specifically at which species are incorrectly and correctly predicted
species_list = list(df.index)
print('Incorrectly IDed  True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] != y_pred[i]:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
print('Correctly IDed   True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] == 1 and y_pred[i] == 1:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
    

header = list(input_ed.columns)

# define the model again to re-test on a held out dataset, usually have  subsample=0.9, colsample_bynode=0.8, 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
xtrain, xtest, ytrain, ytest=train_test_split(ml_input, output, test_size=0.10)
class_weights_training = compute_sample_weight('balanced', ytrain)
#sample_weight=class_weights_training
#model.fit(xtrain, ytrain) 
model.fit(xtrain, ytrain, sample_weight=class_weights_training) 

print('Mini_test')
ypred = model.predict(xtest)
cm = confusion_matrix(ytest,ypred)
total = sum(cm[0]) + sum(cm[1])
mis = cm[0][1] + cm[1][0]
print('Misclassification rate:', mis/total)

print(cm)

plt.figure()
plot_confusion_matrix(cm, classes=['0','1'],
                     title='Confusion matrix, without normalization')


#rebuild model yet again for the final big cross validation accuracy metrics with RepeatedStratifiedKFold
classes_weights = compute_sample_weight('balanced', output)
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
sample_weight=classes_weights
model.fit(ml_input, output, sample_weight=classes_weights)
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, ml_input, output, scoring='balanced_accuracy', cv=cv)
print(n_scores)
# report performance
print('Balanced Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)), '\n')


#report feature importances in order of most important to least
importance_list = []
importances = list(model.feature_importances_)
o = 0
m = 0
top10 = []
for i in model.feature_importances_:
    importance_list.append([i, header[o]])
    o = o + 1
for i in importance_list:
    if i[0] == max(importances):
        print(i[1], '\t', i[0])
        if m < 10:
            top10.append(i[1])
            m = m + 1
            importances.remove(i[0])
        else:
            importances.remove(i[0])
    else:
        importance_list.append(i)
        

In [ ]:
#ROC_AUC code 
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBRFClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.utils import compute_sample_weight
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics as metrics


df1 = pd.read_csv('final_KEGG_carb.csv', index_col=0)
df2 = pd.read_csv('Y1000KEGG_m.csv', index_col=0)
df = pd.concat([df1, df2.set_index(df1.index)], axis=1)

predicting = 'Carb_FINAL'
df = df.dropna(subset=[predicting])
df = df[df[predicting] != 0]
df[predicting] = df[predicting].replace([2],0)

input_ed = df.drop(['Carb_FINAL'], axis=1)

X = np.array(input_ed) #array of input
y = np.array(df[predicting])

from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve

# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=10)
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs=8)
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
tprs = []
base_fpr = np.linspace(0, 1, 101)

    
plt.figure(figsize=(5, 5))
plt.axes().set_aspect('equal', 'datalim')

for i, (train, test) in enumerate(cv.split(X, y)):
    model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss',  n_jobs=8)
    model.fit(X[train], y[train])
    y_score = model.predict_proba(X[test])
    fpr, tpr, _ = roc_curve(y[test], y_score[:, 1])
    roc_auc = metrics.auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, 'b', alpha=0.15)
    tpr = np.interp(base_fpr, fpr, tpr)
    tpr[0] = 0.0
    tprs.append(tpr)

tprs = np.array(tprs)
mean_tprs = tprs.mean(axis=0)
aucs = np.array(aucs)
mean_auc = aucs.mean(axis=0)
std = tprs.std(axis=0)

tprs_upper = np.minimum(mean_tprs + std, 1)
tprs_lower = mean_tprs - std


plt.plot(base_fpr, mean_tprs, 'b', label = 'Mean ROC (AUC = %0.2f)' % mean_auc )
plt.legend(loc = 'lower right')
plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3, label=r"$\pm$ 1 std. dev.")
plt.legend(loc = 'lower right')

plt.plot([0, 1], [0, 1],'r--', label="Chance")
plt.legend(loc = 'lower right')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig("gen_spec.png", dpi=500)
plt.show()

In [ ]:
#function to recursively run the random forest while dropping the 20 most important traits to see how many traits are truly important
def randomforest(file1, file2, predicting, drop):
    import pandas as pd
    df1 = pd.read_csv(file1, index_col=0)
    df2 = pd.read_csv(file2, index_col=0)
    df = pd.concat([df1, df2.set_index(df1.index)], axis=1)
    import numpy as np
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import RepeatedStratifiedKFold
    from xgboost import XGBRFClassifier
    from sklearn.metrics import confusion_matrix
    from sklearn.model_selection import train_test_split
    import xgboost as xgb
    from sklearn.utils import compute_sample_weight
    import matplotlib.pyplot as plt
    df = df.dropna(subset=[predicting])
    df = df[df[predicting] != 0]
    df[predicting] = df[predicting].replace([2],0)
    input_ed = df.drop(drop, axis=1)
    ml_input = np.array(input_ed) #array of input
    output = np.array(df[predicting]) #array of output
    model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
    header = list(input_ed.columns)
    xtrain, xtest, ytrain, ytest=train_test_split(ml_input, output, test_size=0.10)
    class_weights_training = compute_sample_weight('balanced', ytrain)
    model.fit(xtrain, ytrain, sample_weight=class_weights_training) 
    #ypred = model.predict(xtest)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores = cross_val_score(model, ml_input, output, scoring='balanced_accuracy', cv=cv)
    print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)), '\n')
    importance_list = []
    importances = list(model.feature_importances_)
    o = 0
    m = 0
    top2 = []
    for i in model.feature_importances_:
        importance_list.append([i, header[o]])
        o = o + 1
    for i in importance_list:
        if i[0] == max(importances):
            if m < 20:
                print(i[1], '\t', i[0])
                top2.append(i[1])
                m = m + 1
                importances.remove(i[0])
            else:
                break
        else:
            importance_list.append(i)
    for i in top2:
        drop.append(i)
    randomforest(file1,file2,file3,predicting,drop)

In [ ]:
#running the recursive function
randomforest('Y1000KEGG_m.csv','final_KEGG_carb.csv','Carb_FINAL', ['Carb_FINAL'])